# Catch 4 Critters EEG - ERP analysis

In [ ]:
# Import relevant packages
import os
import numpy as np
import mne
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
import glm

In [ ]:
# Define data file locations etc
# Define data file locations etc
homeDir = '/Users/yvonnefonken/Documents/Oxford/Research Projects/intracranial'
os.chdir(homeDir)

# set montage file
#montage = homeDir + "montage/m.elp"
subjects = ['UCL01', 'UCL02'] # Only analyzing 18-30 for now

filename = 'dataClean-epo.fif'

#filenames = ['','' ,'' ,'','', '','' ,'' ,'','', '','' ,'' ,'','', '', '', 'testClean-epo.fif', 'testClean-epo.fif','testClean-epo.fif', 'testClean-epo.fif',
#            'testClean-epo.fif', 'testClean-epo.fif','testClean-epo.fif', 'testClean-epo.fif','testClean-epo.fif', 'testClean-epo.fif','testClean-epo.fif', 'testClean-epo.fif',
            #'testClean-epo.fif', 'testClean-epo.fif','testClean-epo.fif', 'testClean-epo.fif','testClean-epo.fif', 'testClean-epo.fif']

In [ ]:
datafile = os.path.join(homeDir, subjects[1], filename)
epochs = mne.read_epochs(datafile, preload=True)

epochsDummy = epochs.copy()
epochsDummy.resample(epochs.info['sfreq']/10)
epochsDummy._data = []


In [ ]:
epochs.resample?

In [ ]:
plt.plot(epochs._data[7,80,:]-epochs._data[7,81,:])

In [ ]:
plt.plot(epochsBip._data[7,80,:])

In [ ]:
def bipolarRef(epochs):
    
    epochs2 = epochs.copy()
    epochs2._data = []
    data = epochs._data
    dataBip = np.zeros((data.shape))
    chns = epochs.info['ch_names']
    elecInd = 0
    
    for i in range(0, len(chns)-1):
        for j in range(0, data.shape[0]): # trials
            
            if i < len(chns):
                if chns[i][:6] == chns[i+1][:6]:
                    # calculate bipolar diff
                    dataBip[j,i,:] = data[j,i,:]- data[j,i+1,:]
                else:
                    # insert zeros 
                    dataBip[j,i,:] = np.zeros((1,1, data.shape[2]))
            else:
                    # insert zeros 
                    dataBip[j,i,:] = np.zeros((1,1, data.shape[2]))

    epochs2._data = dataBip
    
    return epochs2

In [ ]:
# Reorient to RT's
# RT's in metadata are per event
# i.e. RTselec1 is time since critter appeared under the scene (3.3s + RTselec1 in trialtime)
# RTplace1: 3.3s + RTselec1 + RTplace1
# etc
frequencies = np.logspace(*np.log10([1, 150]), num = 40) 
frequencies

In [ ]:
from mne.time_frequency import tfr_morlet
tfr_morlet?

In [ ]:
epochs.metadata['RTselec1']

In [ ]:
# load data
#subs = [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30]
# or all subs
subs = [2]
bipolar = True
wmRef = False
chanSelec = True

TFR = False
# TFR parameters
decim = 10
frequencies = np.logspace(*np.log10([1, 150]), num = 40)  # define frequencies of interest
n_cycles = 4. 

evokedL_dictAll= dict()
evokedO_dictAll= dict()
chans = ['EEG laH', 'EEG lpH', 'EEG raH', 'EEG rpH']
picks = []


for i in subs:
    subInd = i - 1
    
    
    evokedL_dict= dict()
    evokedO_dict= dict()

    #load data
    datafile = os.path.join(homeDir, subjects[subInd], filename)
    epochs = mne.read_epochs(datafile, preload=True)
    
    if bipolar: 
        epochs = bipolarRef(epochs)
    elif wmRef:
        print('insert code')
        
    if chanSelec: 
        for j in chans:
            for k in range(len(epochs.info['ch_names'])):
                if j in epochs.info['ch_names'][k]:
                    picks.append(k)
        
    if TFR:
        from mne.time_frequency import tfr_morlet
        epochs = tfr_morlet(epochs, freqs=frequencies,n_cycles=n_cycles, picks = picks, use_fft=False, decim=decim, zero_mean=True, average = False, return_itc = False)
    else:
        # Filter
        epochs.filter(None,20, picks = 'all', fir_design = 'firwin')
    
    # Separate different conditions 
    indO = epochs.metadata['BlockType'] == 'Orienting'
    indL = (epochs.metadata['BlockType'] == 'Learning') # take only trials with at least one exposure
    indEarly = epochs.metadata['Delay'] < 2300
    indLate = epochs.metadata['Delay'] > 2300
    indRand = epochs.metadata['Condition'] == 'random'
    indPred = epochs.metadata['Condition'] == 'predicted'
    indLeft = epochs.metadata['Target1X'] < 0
    indRight = epochs.metadata['Target1X'] > 0
    indLeft1 = epochs.metadata['Target1X'] < 0
    indRight1 = epochs.metadata['Target1X'] > 0
    #indRespL = epochs.metadata['Answer'] < 6
    #indRespR = epochs.metadata['Answer'] > 5
    indCorrect = epochs.metadata['correct'] == 1
    indnRep1 = epochs.metadata['nRep'] == 1
    indnRep2 = epochs.metadata['nRep'] == 2
    indnRep3 = epochs.metadata['nRep'] == 3
    indnRep4 = epochs.metadata['nRep'] == 4
    indnRep5 = epochs.metadata['nRep'] == 5
    indnRep6 = epochs.metadata['nRep'] == 6
    indnRep7 = epochs.metadata['nRep'] == 7
    indnRep8 = epochs.metadata['nRep'] == 8
    
    
    # separate conditions by place of visual target
    evokedO_dict['Early/Random/Left'] = epochs[indO*indEarly*indRand*indLeft]
    evokedO_dict['Early/Random/Right'] = epochs[indO*indEarly*indRand*indRight]
    evokedO_dict['Late/Random/Left'] = epochs[indO*indLate*indRand*indLeft]
    evokedO_dict['Late/Random/Right'] = epochs[indO*indLate*indRand*indRight]
    evokedO_dict['Early/Predicted/Left'] = epochs[indO*indEarly*indPred*indLeft]
    evokedO_dict['Early/Predicted/Right'] = epochs[indO*indEarly*indPred*indRight]
    evokedO_dict['Late/Predicted/Left'] = epochs[indO*indLate*indPred*indLeft]
    evokedO_dict['Late/Predicted/Right'] =epochs[indO*indLate*indPred*indRight]
    
    # Separate random/predicted and early/late only
    evokedO_dict['Early/Random'] = epochs[indO*indEarly*indRand]
    evokedO_dict['Late/Random'] = epochs[indO*indLate*indRand]
    evokedO_dict['Early/Predicted'] = epochs[indO*indEarly*indPred]
    evokedO_dict['Late/Predicted'] = epochs[indO*indLate*indPred]
    
    
    
    evokedL_dict['Random/Left'] = epochs[indL*indRand*indLeft1] # Left animal first
    evokedL_dict['Random/Right'] = epochs[indL*indRand*indRight1]
    evokedL_dict['Predicted/Left'] = epochs[indL*indPred*indLeft1] # Left animal first
    evokedL_dict['Predicted/Right'] = epochs[indL*indPred*indRight1]
    
    evokedL_dict['Random/nRep1'] = epochs[indL*indRand*indnRep1]
    evokedL_dict['Predicted/nRep1'] = epochs[indL*indPred*indnRep1]
    evokedL_dict['Random/nRep2'] = epochs[indL*indRand*indnRep2]
    evokedL_dict['Predicted/nRep2'] = epochs[indL*indPred*indnRep2]
    evokedL_dict['Random/nRep3'] = epochs[indL*indRand*indnRep3]
    evokedL_dict['Predicted/nRep3'] = epochs[indL*indPred*indnRep3]
    evokedL_dict['Random/nRep4'] = epochs[indL*indRand*indnRep4]
    evokedL_dict['Predicted/nRep4'] = epochs[indL*indPred*indnRep4]
    evokedL_dict['Random/nRep5'] = epochs[indL*indRand*indnRep5]
    evokedL_dict['Predicted/nRep5'] = epochs[indL*indPred*indnRep5]
    evokedL_dict['Random/nRep6'] = epochs[indL*indRand*indnRep6]
    evokedL_dict['Predicted/nRep6'] = epochs[indL*indPred*indnRep6]
    evokedL_dict['Random/nRep7'] = epochs[indL*indRand*indnRep7]
    evokedL_dict['Predicted/nRep7'] = epochs[indL*indPred*indnRep7]
    evokedL_dict['Random/nRep8'] = epochs[indL*indRand*indnRep8]
    evokedL_dict['Predicted/nRep8'] = epochs[indL*indPred*indnRep8]
    
    evokedL_dict['nRep1'] = epochs[indL*indnRep1]
    evokedL_dict['nRep2'] = epochs[indL*indnRep2]
    evokedL_dict['nRep3'] = epochs[indL*indnRep3]
    evokedL_dict['nRep4'] = epochs[indL*indnRep4]
    evokedL_dict['nRep5'] = epochs[indL*indnRep5]
    evokedL_dict['nRep6'] = epochs[indL*indnRep6]
    
    evokedL_dict['Random'] = epochs[indL*indRand]
    evokedL_dict['Predicted'] = epochs[indL*indPred]
    
    
    
    #evokedO_dictAll['S' + str(i)] = evokedO_dict
    #evokedL_dictAll['S' + str(i)] = evokedL_dict
    
    #indRandEarlyL = epochs.average()
   # indRandEarlyR = 
    #indRandLateL = 
   # indRandLateR = 
   # indPredEarlyL = 
   # indPredEarlyR = 
   # indPredLateL = 
   # indPredLateR = 
    
    

In [ ]:
# Save TFR data
fileName = 'dataTFR-bipolarHPC.fif'
saveFile = os.path.join(homeDir, subjects[subInd], fileName)
epochs.save(saveFile, overwrite = True)

In [7]:
#del epochs
epochs = mne.read_epochs(os.path.join(homeDir, subjects[1], 'dataTFR-bipolarHPC.fif'))

# Separate different conditions 
indO = epochs.metadata['BlockType'] == 'Orienting'
indL = (epochs.metadata['BlockType'] == 'Learning') # take only trials with at least one exposure
indEarly = epochs.metadata['Delay'] < 2300
indLate = epochs.metadata['Delay'] > 2300
indRand = epochs.metadata['Condition'] == 'random'
indPred = epochs.metadata['Condition'] == 'predicted'
indLeft = epochs.metadata['Target1X'] < 0
indRight = epochs.metadata['Target1X'] > 0
indLeft1 = epochs.metadata['Target1X'] < 0
indRight1 = epochs.metadata['Target1X'] > 0
#indRespL = epochs.metadata['Answer'] < 6
#indRespR = epochs.metadata['Answer'] > 5
indCorrect = epochs.metadata['correct'] == 1
indnRep1 = epochs.metadata['nRep'] == 1
indnRep2 = epochs.metadata['nRep'] == 2
indnRep3 = epochs.metadata['nRep'] == 3
indnRep4 = epochs.metadata['nRep'] == 4
indnRep5 = epochs.metadata['nRep'] == 5
indnRep6 = epochs.metadata['nRep'] == 6
indnRep7 = epochs.metadata['nRep'] == 7
indnRep8 = epochs.metadata['nRep'] == 8


# separate conditions by place of visual target
evokedO_dict['Early/Random/Left'] = epochs[indO*indEarly*indRand*indLeft]
evokedO_dict['Early/Random/Right'] = epochs[indO*indEarly*indRand*indRight]
evokedO_dict['Late/Random/Left'] = epochs[indO*indLate*indRand*indLeft]
evokedO_dict['Late/Random/Right'] = epochs[indO*indLate*indRand*indRight]
evokedO_dict['Early/Predicted/Left'] = epochs[indO*indEarly*indPred*indLeft]
evokedO_dict['Early/Predicted/Right'] = epochs[indO*indEarly*indPred*indRight]
evokedO_dict['Late/Predicted/Left'] = epochs[indO*indLate*indPred*indLeft]
evokedO_dict['Late/Predicted/Right'] =epochs[indO*indLate*indPred*indRight]

# Separate random/predicted and early/late only
evokedO_dict['Early/Random'] = epochs[indO*indEarly*indRand]
evokedO_dict['Late/Random'] = epochs[indO*indLate*indRand]
evokedO_dict['Early/Predicted'] = epochs[indO*indEarly*indPred]
evokedO_dict['Late/Predicted'] = epochs[indO*indLate*indPred]



evokedL_dict['Random/Left'] = epochs[indL*indRand*indLeft1] # Left animal first
evokedL_dict['Random/Right'] = epochs[indL*indRand*indRight1]
evokedL_dict['Predicted/Left'] = epochs[indL*indPred*indLeft1] # Left animal first
evokedL_dict['Predicted/Right'] = epochs[indL*indPred*indRight1]

evokedL_dict['Random/nRep1'] = epochs[indL*indRand*indnRep1]
evokedL_dict['Predicted/nRep1'] = epochs[indL*indPred*indnRep1]
evokedL_dict['Random/nRep2'] = epochs[indL*indRand*indnRep2]
evokedL_dict['Predicted/nRep2'] = epochs[indL*indPred*indnRep2]
evokedL_dict['Random/nRep3'] = epochs[indL*indRand*indnRep3]
evokedL_dict['Predicted/nRep3'] = epochs[indL*indPred*indnRep3]
evokedL_dict['Random/nRep4'] = epochs[indL*indRand*indnRep4]
evokedL_dict['Predicted/nRep4'] = epochs[indL*indPred*indnRep4]
evokedL_dict['Random/nRep5'] = epochs[indL*indRand*indnRep5]
evokedL_dict['Predicted/nRep5'] = epochs[indL*indPred*indnRep5]
evokedL_dict['Random/nRep6'] = epochs[indL*indRand*indnRep6]
evokedL_dict['Predicted/nRep6'] = epochs[indL*indPred*indnRep6]
evokedL_dict['Random/nRep7'] = epochs[indL*indRand*indnRep7]
evokedL_dict['Predicted/nRep7'] = epochs[indL*indPred*indnRep7]
evokedL_dict['Random/nRep8'] = epochs[indL*indRand*indnRep8]
evokedL_dict['Predicted/nRep8'] = epochs[indL*indPred*indnRep8]

evokedL_dict['nRep1'] = epochs[indL*indnRep1]
evokedL_dict['nRep2'] = epochs[indL*indnRep2]
evokedL_dict['nRep3'] = epochs[indL*indnRep3]
evokedL_dict['nRep4'] = epochs[indL*indnRep4]
evokedL_dict['nRep5'] = epochs[indL*indnRep5]
evokedL_dict['nRep6'] = epochs[indL*indnRep6]

evokedL_dict['Random'] = epochs[indL*indRand]
evokedL_dict['Predicted'] = epochs[indL*indPred]


Reading /Users/yvonnefonken/Documents/Oxford/Research Projects/intracranial/UCL02/dataTFR-bipolarHPC.fif ...


<ipython-input-7-b55c8bea3815>:2: RuntimeWarning: This filename (/Users/yvonnefonken/Documents/Oxford/Research Projects/intracranial/UCL02/dataTFR-bipolarHPC.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(os.path.join(homeDir, subjects[1], 'dataTFR-bipolarHPC.fif'))


ValueError: file does not start with a file id tag

In [ ]:
fileName

In [ ]:
chan = 'EEG rpH3'
condition = 'Predicted'
vmax = 500
vmin = -1*vmax

# Parameters for plotting
rtS1 = evokedL_dict[condition].metadata['RTselec1']/1000.0 + 3.3
orderS1 = rtS1.argsort()  
rtP1 = evokedL_dict[condition].metadata['RTselec1']/1000.0 + evokedL_dict[condition].metadata['RTplace1']/1000.0 + 3.3
orderP1 = rtP1.argsort() 

rtS2 = evokedL_dict[condition].metadata['RTselec1']/1000.0 +evokedL_dict[condition].metadata['RTplace1']/1000.0 + evokedL_dict[condition].metadata['RTselec2']/1000.0+ 3.3
orderS2 = rtS2.argsort()  
rtP2 = evokedL_dict[condition].metadata['RTselec1']/1000.0 + evokedL_dict[condition].metadata['RTplace1']/1000.0 + evokedL_dict[condition].metadata['RTselec2']/1000.0 + evokedL_dict[condition].metadata['RTplace2']/1000.0 + 3.3
orderP2 = rtP2.argsort() 

picks = evokedL_dict[condition].ch_names.index(chan)

# Plot sorted data 
evokedL_dict[condition].plot_image(picks = picks, order=orderS1, sigma=0.,
                  overlay_times=rtS1, vmin = vmin, vmax = vmax)

evokedL_dict[condition].plot_image(picks = picks, order=orderP1, sigma=0.,
                  overlay_times=rtP1, vmin = vmin, vmax = vmax)

evokedL_dict[condition].plot_image(picks = picks, order=orderS2, sigma=0.,
                  overlay_times=rtS2, vmin = vmin, vmax = vmax)

evokedL_dict[condition].plot_image(picks = picks, order=orderP2, sigma=0.,
                  overlay_times=rtP2, vmin = vmin, vmax = vmax)

In [ ]:
#chan = 'EEG lpH1'
condition = 'Early/Predicted'

vmax = 300
vmin = -1*vmax

# Parameters for plotting
if 'Early'in condition:
    delay = 0.8
elif 'Late' in condition:
    delay = 2.0

# Parameters for plotting
rtS1 = evokedO_dict[condition].metadata['RTselec1']/1000.0 + delay
orderS1 = rtS1.argsort()  


picks = evokedO_dict[condition].ch_names.index(chan)

# Plot sorted data 
evokedO_dict[condition].plot_image(picks = picks, order=orderS1, sigma=0.,
                  overlay_times=rtS1, vmin =vmin, vmax = vmax)



In [ ]:
#evokedL_dict['Predicted'].times
#epochsDummy.times = []
epochsDummy = epochsDummy._replace(times = evokedL_dict[condition].times)

In [ ]:
evokedL_dict[condition]._data.shape

In [ ]:
epochsDummy._data.shape

In [ ]:
picks

In [ ]:
# For TFR - hack into epochs.plot_image for TF timecourses -- LEARNING


chan = 'EEG rpH2'
condition = 'Predicted'
sfreq = [55., 150.]
freqInd = (frequencies <sfreq[1]) & (frequencies >sfreq[0])
vmax = 0.01

#evokedL_dict[condition].apply_baseline(baseline = [-1.5, -1.1], mode = 'ratio')

epochsDummy._data = np.mean(evokedL_dict[condition]._data[:,:,freqInd,:], axis = 2)
#delattr( epochsDummy, 'times')
#epochsDummy.times = evokedL_dict[condition].times

# Parameters for plotting
rtS1 = evokedL_dict[condition].metadata['RTselec1']/1000.0 + 3.3
orderS1 = rtS1.argsort()  
rtP1 = evokedL_dict[condition].metadata['RTselec1']/1000.0 + evokedL_dict[condition].metadata['RTplace1']/1000.0 + 3.3
orderP1 = rtP1.argsort() 

rtS2 = evokedL_dict[condition].metadata['RTselec1']/1000.0 +evokedL_dict[condition].metadata['RTplace1']/1000.0 + evokedL_dict[condition].metadata['RTselec2']/1000.0+ 3.3
orderS2 = rtS2.argsort()  
rtP2 = evokedL_dict[condition].metadata['RTselec1']/1000.0 + evokedL_dict[condition].metadata['RTplace1']/1000.0 + evokedL_dict[condition].metadata['RTselec2']/1000.0 + evokedL_dict[condition].metadata['RTplace2']/1000.0 + 3.3
orderP2 = rtP2.argsort() 

picks = evokedL_dict[condition].ch_names.index(chan)

# Plot sorted data 
epochsDummy.plot_image(picks = picks, order=orderS1, sigma=0.,
                  overlay_times=rtS1, vmax = vmax)

epochsDummy.plot_image(picks = picks, order=orderP1, sigma=0.,
                  overlay_times=rtP1, vmax = vmax)

epochsDummy.plot_image(picks = picks, order=orderS2, sigma=0.,
                  overlay_times=rtS2, vmax = vmax)

epochsDummy.plot_image(picks = picks, order=orderP2, sigma=0.,
                  overlay_times=rtP2, vmax = vmax)

In [ ]:
epochsDummy.plot_image?

In [ ]:
# For TFR - hack into epochs.plot_image for TF timecourses -- Orienting


chan = 'EEG rpH1'
condition = 'Early/Predicted'
sfreq = [60., 150.]
freqInd = (frequencies <sfreq[1]) & (frequencies >sfreq[0])
vmax = 0.01

#evokedL_dict[condition].apply_baseline(baseline = [-1.5, -1.1], mode = 'ratio')

epochsDummy._data = np.mean(evokedO_dict[condition]._data[:,:,freqInd,:], axis = 2)
#delattr( epochsDummy, 'times')
#epochsDummy.times = evokedL_dict[condition].times

# Parameters for plotting
if 'Early'in condition:
    delay = 0.8
elif 'Late' in condition:
    delay = 2.0
    
rtS1 = evokedO_dict[condition].metadata['RTselec1']/1000.0 + delay
orderS1 = rtS1.argsort()  

picks = evokedO_dict[condition].ch_names.index(chan)

# Plot sorted data 
epochsDummy.plot_image(picks = picks, order=orderS1, sigma=0.,
                  overlay_times=rtS1, vmax = vmax)



In [ ]:
evokedO_dict

In [ ]:
evokedL_dict['Random'].apply_baseline(baseline = [-1.5, -1.1], mode = 'percent')

In [ ]:
epochs.plot_image?

In [ ]:
# Plot individual electrodes

ElecGroup = 'EEG laH'
tWin = [-2., 4.]

condition1 = 'Random'
condition2 = 'Predicted'



colors = ['CornFlowerBlue', 'lightblue', 'Crimson', 'coral' ]
#linestyles = dict(Left = '-', Right = '--')
#colors = dict(Predicted = "Crimson", Random = "CornFlowerBlue")

fig, axes = plt.subplots(nrows=12, ncols=1, figsize = (20,40))
fig.subplots_adjust(hspace=0.5)
fig.suptitle('ERPs Learning block')


timeTicks = evokedL_dict['Random/Left'].times
tInd = (timeTicks > tWin[0]) & (timeTicks < tWin[1])
axNum = 0

for i in evokedL_dict['Random/Left'].ch_names: 
    
    
    # Plot data
    if ElecGroup in i:
        pick = evokedL_dict["Random/Left"].ch_names.index(i)
         
        sb.tsplot (data = evokedL_dict[condition1]._data[:,pick, tInd], time = timeTicks[tInd], ax = axes[axNum], color = colors[0], linestyle = '-')
        sb.tsplot (data = evokedL_dict[condition2]._data[:,pick, tInd], time = timeTicks[tInd], ax = axes[axNum], color = colors[2],linestyle = '-')
                    
        axes[axNum].set(title = 'Electrode ' + i)
        axes[axNum].axvline(x = 0., color = 'black')
        axes[axNum].axvline(x = 0.8, color = 'Gray')
        axes[axNum].axvline(x = 2., color = 'Gray')
        axes[axNum].axvline(x = -1.2, color = 'black', linestyle = ':')
        axes[axNum].axvline(x = 3.3, color = 'black', linestyle = ':')
        axNum += 1




In [ ]:
# Plot individual electrodes

ElecGroup = 'EEG rpH'
tWin = [-2., 4.]

condition1 = 'Predicted/nRep1'
condition2 = 'Predicted/nRep2'
condition3 = 'Predicted/nRep3'
condition4 = 'Predicted/nRep4'
condition5 = 'Predicted/nRep5'
condition6 = 'Predicted/nRep6'

colors = dict(nRep1 = 'Gainsboro', nRep2 = 'LightGray', nRep3 = 'Silver', nRep4 = 'DarkGray', nRep5 = 'Gray', nRep6 = 'DimGray' )


colors = ['Gainsboro', 'LightGray', 'Silver', 'DarkGray', 'Gray', 'DimGray' ]
#linestyles = dict(Left = '-', Right = '--')
#colors = dict(Predicted = "Crimson", Random = "CornFlowerBlue")

fig, axes = plt.subplots(nrows=12, ncols=1, figsize = (20,40))
fig.subplots_adjust(hspace=0.5)
fig.suptitle('ERPs Learning block')


timeTicks = evokedL_dict['Random/Left'].times
tInd = (timeTicks > tWin[0]) & (timeTicks < tWin[1])

axNum = 0

for i in evokedL_dict['Random/Left'].ch_names: 
    
    
    # Plot data
    if ElecGroup in i:
        pick = evokedL_dict["Random/Left"].ch_names.index(i)
         
        sb.tsplot (data = evokedL_dict[condition1]._data[:,pick, tInd], time = timeTicks[tInd], ax = axes[axNum], color = colors[0])
        sb.tsplot (data = evokedL_dict[condition2]._data[:,pick, tInd], time = timeTicks[tInd], ax = axes[axNum], color = colors[1])
        sb.tsplot (data = evokedL_dict[condition3]._data[:,pick, tInd], time =timeTicks[tInd], ax = axes[axNum], color = colors[2])
        sb.tsplot (data = evokedL_dict[condition4]._data[:,pick, tInd], time = timeTicks[tInd], ax = axes[axNum], color = colors[3])
        sb.tsplot (data = evokedL_dict[condition5]._data[:,pick, tInd], time = timeTicks[tInd], ax = axes[axNum], color = colors[4])
        sb.tsplot (data = evokedL_dict[condition6]._data[:,pick, tInd], time = timeTicks[tInd], ax = axes[axNum], color = colors[5])    
            
        axes[axNum].set(title = 'Electrode ' + i)
        axes[axNum].axvline(x = 0., color = 'black')
        axes[axNum].axvline(x = 0.8, color = 'Gray')
        axes[axNum].axvline(x = 2., color = 'Gray')
        axes[axNum].axvline(x = -1.2, color = 'black', linestyle = ':')
        axes[axNum].axvline(x = 3.3, color = 'black', linestyle = ':')
        axNum += 1





In [ ]:
# Plot individual electrodes

Elec = 'EEG rpH3'
tWin = [-2., 4.]

condition1 = 'Predicted'
condition2 = 'Random'


#colors = dict(nRep1 = 'Gainsboro', nRep2 = 'LightGray', nRep3 = 'Silver', nRep4 = 'DarkGray', nRep5 = 'Gray', nRep6 = 'DimGray' )


#colors = ['Gainsboro', 'LightGray', 'Silver', 'DarkGray', 'Gray', 'DimGray', 'black' ]
#linestyles = dict(Left = '-', Right = '--')
#colors = dict(Predicted = "Crimson", Random = "CornFlowerBlue")

fig, axes = plt.subplots(nrows=8, ncols=1, figsize = (20,40))
fig.subplots_adjust(hspace=0.5)
fig.suptitle('ERPs Learning block')


timeTicks = evokedL_dict['Random/Left'].times
tInd = (timeTicks > tWin[0]) & (timeTicks < tWin[1])

axNum = 0
pick = evokedL_dict["Random/Left"].ch_names.index(Elec)

for i in range(1,9):

    nRep = '/nRep' + str(i)


    sb.tsplot (data = evokedL_dict[condition1 + nRep]._data[:,pick, tInd], time = timeTicks[tInd], ax = axes[axNum], color = 'Crimson')
    sb.tsplot (data = evokedL_dict[condition2 + nRep]._data[:,pick, tInd], time = timeTicks[tInd], ax = axes[axNum], color = 'CornFlowerBlue')

    axes[axNum].set(title = 'Electrode ' + Elec + nRep)
    axes[axNum].axvline(x = 0., color = 'black')
    axes[axNum].axvline(x = 0.8, color = 'Gray')
    axes[axNum].axvline(x = 2., color = 'Gray')
    axes[axNum].axvline(x = -1.2, color = 'black', linestyle = ':')
    axes[axNum].axvline(x = 3.3, color = 'black', linestyle = ':')
    axNum += 1






In [ ]:
# Plot individual electrodes ORIENTING

ElecGroup = 'EEG lpH'
tWin = [-2., 4.]

condition1 = 'Early/Random'
condition2 = 'Late/Random'
condition3 = 'Early/Predicted'
condition4 = 'Late/Predicted'


colors = ['CornFlowerBlue', 'lightblue', 'Crimson', 'coral' ]
#linestyles = dict(Left = '-', Right = '--')
#colors = dict(Predicted = "Crimson", Random = "CornFlowerBlue")

fig, axes = plt.subplots(nrows=10, ncols=1, figsize = (20,40))
fig.subplots_adjust(hspace=0.5)
fig.suptitle('ERPs Learning block')


timeTicks = evokedO_dict['Early/Random'].times
tInd = (timeTicks > tWin[0]) & (timeTicks < tWin[1])

axNum = 0

for i in evokedO_dict['Early/Random'].ch_names: 
    
    
    # Plot data
    if ElecGroup in i:
        pick = evokedO_dict["Early/Random"].ch_names.index(i)
         
        sb.tsplot (data = evokedO_dict[condition1]._data[:,pick, tInd], time = timeTicks[tInd], ax = axes[axNum], color = colors[0], linestyle = '-')
        sb.tsplot (data = evokedO_dict[condition2]._data[:,pick, tInd], time = timeTicks[tInd], ax = axes[axNum], color = colors[0], linestyle = '--')
        sb.tsplot (data = evokedO_dict[condition3]._data[:,pick, tInd], time = timeTicks[tInd], ax = axes[axNum], color = colors[2],linestyle = '-')
        sb.tsplot (data = evokedO_dict[condition4]._data[:,pick, tInd], time = timeTicks[tInd], ax = axes[axNum], color = colors[2], linestyle = '--')

            
        axes[axNum].set(title = 'Electrode ' + i)
        axes[axNum].axvline(x = 0., color = 'black')
        axes[axNum].axvline(x = 0.8, color = 'Gray')
        axes[axNum].axvline(x = 2., color = 'Gray')
        axes[axNum].axvline(x = -1.2, color = 'black', linestyle = ':')
        axes[axNum].axvline(x = 3.3, color = 'black', linestyle = ':')
        axNum += 1



In [ ]:
epochs.ch_names

In [ ]:
len(epochs.metadata)

## Do GLM analysis on P300 amplitude

In [ ]:
# extract P300 amplitude per trial

elec = 'EEG raH3'

P3time1 = [1.1, 1.4]
P3time2 = [2.3, 2.6]

tInd1 = (epochs.times > P3time1[0]) & (epochs.times <P3time1[1])
tInd2 = (epochs.times > P3time2[0]) & (epochs.times <P3time2[1])

data1 = []
data2 = []
newDF = []

pick = epochs.ch_names.index(elec)

for i in range(epochs._data.shape[0]):

    data1.append(np.mean(epochs._data[i,pick, tInd1]))
    data2.append(np.mean(epochs._data[i,pick, tInd2]))

newDF = epochs.metadata.copy()
newDF['P3early'] = data1
newDF['P3late'] = data2

In [ ]:
P3o = []
for index, row in newDF.iterrows():
    if row['BlockType'] == 'Learning':
        P3o.append(0.)
    elif row['BlockType'] == 'Orienting':
        if row['Delay'] == 2000.0:
            P3o.append(row['P3early'])
        else: 
            P3o.append(row['P3late'])
newDF['P3'] = P3o
            

In [ ]:
# Remove outliers from data

df = newDF
df['targetDistance1'] = np.sqrt(df['Target1X']*df['Target1X'] + df['Target1Y']*df['Target1Y'])
df['targetDistance2'] = np.sqrt(df['Target2X']*df['Target2X'] + df['Target2Y']*df['Target2Y'])
#newDF['targetDistance'] = np.sqrt(df['TargetX']*df['TargetX'] + df['TargetY']*df['TargetY'])



#Learning first
df.drop(df[df['RTselec1'] > 3000].index, inplace = True)
print(len(df))
df.drop(df[df['RTselec2'] > 3000].index, inplace = True)
print(len(df))
df.drop(df[df['RTplace1'] > 3000].index, inplace = True)
print(len(df))
df.drop(df[df['RTplace2'] > 3000].index, inplace = True)
print(len(df))

df.drop(df[df['pixelError1'] > 200].index, inplace = True)
print(len(df))
df.drop(df[df['pixelError2'] > 200].index, inplace = True)
print(len(df))
df.drop(df[df['pixelError'] > 200].index, inplace = True)
print(len(df))
# remove pixelerrors that are too large


# remove RT's that are too long
#df.drop(df[df['P3early'] < -0.00015].index, inplace = True)
#print(len(df))

# remove P3 values that are too large?

#mu = np.mean(newDF['P3early'])
#sd = np.std(newDF['P3early'])

#newDF.drop(df[df['P3early']])



In [ ]:
sum(df['correct'])

In [ ]:
df = newDF
sum(df[df['pixelError'] > 200].index)

In [ ]:
import statsmodels.formula.api as smf

#dfLME = dfO.dropna(subset = ['pixelError']) # do 'RTselec1' 'MoveOnset' or 'pixelError'
dfLME = newDF[newDF['BlockType'] == 'Learning']
#dfLME['Scene'] = dfLME['Scene'].astype(str)
#dfLME['Delay'] = dfLME['Delay'].astype(str)
#dfLME= dfLME[dfLME['MoveOnset']> 100]


modelLME1 = smf.glm("P3early ~Condition + Scene + pixelError1 + RTselec1 + RTplace1 + correct1 + nRep + correct2 + pixelError2 ",dfLME)
LMERTFit1 = modelLME1.fit()
print(LMERTFit1.summary())

modelLME2 = smf.glm("P3late ~Condition + Scene + pixelError2 + RTselec2 + RTplace2 + correct2 + nRep + correct1 + pixelError1  ",dfLME)
LMERTFit2 = modelLME2.fit()
print('')
print(LMERTFit2.summary())

In [ ]:
#dfLME = dfO.dropna(subset = ['pixelError']) # do 'RTselec1' 'MoveOnset' or 'pixelError'
dfLME = newDF[newDF['BlockType'] == 'Orienting']
#dfLME['Scene'] = dfLME['Scene'].astype(str)
#dfLME['Delay'] = dfLME['Delay'].astype(str)
#dfLME= dfLME[dfLME['MoveOnset']> 100]


modelLME1 = smf.glm("P3 ~Condition + Delay + Condition*Delay + MoveOnset + RTselec1 + MoveOnset + nRep ",dfLME)
LMERTFit1 = modelLME1.fit()
print(LMERTFit1.summary())

In [ ]:
sb.factorplot(data = newDF[newDF['BlockType'] == 'Orienting'], x = 'nRep', y = 'P3', hue = 'Condition', col = 'Delay')

In [ ]:
sb.scatterplot(data = newDF[newDF['BlockType'] == 'Orienting'], x = 'pixelError', y = 'P3', hue = 'Condition')

In [ ]:
sb.factorplot(data = newDF[newDF['BlockType'] == 'Learning'], x = 'nRep', y = 'P3early', hue = 'Condition')
sb.factorplot(data = newDF[newDF['BlockType'] == 'Learning'], x = 'nRep', y = 'P3late', hue = 'Condition')

In [ ]:
sb.factorplot(data = newDF[newDF['BlockType'] == 'Learning'], x = 'Condition', y = 'P3early')
sb.factorplot(data = newDF[newDF['BlockType'] == 'Learning'], x = 'Condition', y = 'P3late')

In [ ]:
sb.scatterplot(data = newDF[newDF['BlockType'] == 'Learning'], x = 'pixelError1', y = 'P3early', hue = 'Condition')
sb.scatterplot(data = newDF[newDF['BlockType'] == 'Learning'], x = 'pixelError2', y = 'P3late', hue = 'Condition')

In [ ]:
sb.boxplot(newDF['P3early'])

In [ ]:

sb.boxplot(data = newDF[newDF['BlockType'] == 'Learning'], x = 'targetDistance1', y = 'P3early', hue = 'Condition')


In [ ]:
# Plot individual electrodes

ElecGroup = 'EEG Am'

condition1 = 'Random/Left'
condition2 = 'Random/Right'
condition3 = 'Predicted/Left'
condition4 = 'Predicted/Right'

colors = ['CornFlowerBlue', 'lightblue', 'Crimson', 'coral' ]
#linestyles = dict(Left = '-', Right = '--')
#colors = dict(Predicted = "Crimson", Random = "CornFlowerBlue")

fig, axes = plt.subplots(nrows=6, ncols=1, figsize = (20,20))
fig.subplots_adjust(hspace=0.5)
fig.suptitle('ERPs Learning block')


timeInd = evokedL_dict['Random/Left'].times

axNum = 0

for i in evokedL_dict['Random/Left'].ch_names: 
    
    
    # Plot data
    if ElecGroup in i:
        pick = evokedL_dict["Random/Left"].ch_names.index(i)
         
        sb.tsplot (data = evokedL_dict[condition1]._data[:,pick, :], time = timeInd, ax = axes[axNum], color = colors[0], linestyle = '-')
        sb.tsplot (data = evokedL_dict[condition2]._data[:,pick, :], time = timeInd, ax = axes[axNum], color = colors[0],linestyle = '--')
        sb.tsplot (data = evokedL_dict[condition3]._data[:,pick, :], time = timeInd, ax = axes[axNum], color = colors[2],linestyle = '-')
        sb.tsplot (data = evokedL_dict[condition4]._data[:,pick, :], time = timeInd, ax = axes[axNum], color = colors[2], linestyle = '--')
                    
        axes[axNum].set(title = 'Electrode ' + i)
        axes[axNum].axvline(x = 0., color = 'black')
        axes[axNum].axvline(x = 0.8, color = 'Gray')
        axes[axNum].axvline(x = 2., color = 'Gray')
        axes[axNum].axvline(x = -1.2, color = 'black', linestyle = ':')
        axes[axNum].axvline(x = 3., color = 'black', linestyle = ':')
        axNum += 1



In [ ]:
# Look at individual data:
subNum = '1'
channel1 = 'EEG STG6'
channel2 = 'EEG STG6'
#evoked_dummy = evokedO_dictAll['S' + str(subNum)]



#Plot specific conditions orienting
evoked_dict= dict()
evoked_dict['Predicted/Early'] = evokedO_dict['Early/Predicted'] # Do right minus left?

evoked_dict['Predicted/Late'] = evokedO_dict['Late/Predicted']

evoked_dict['Random/Early'] = evokedO_dict['Early/Random']
evoked_dict['Random/Late'] = evokedO_dict['Late/Random']



pick = evoked_dict["Predicted/Early"].ch_names.index(channel1)

linestyles = dict(Early = '-', Late = '--')
colors = dict(Predicted = "Crimson", Random = "CornFlowerBlue")

xlim = (-0.2, 2.)

mne.viz.plot_compare_evokeds(evoked_dict, picks=pick, colors=colors,
                             linestyles=linestyles, split_legend=True)



# Plot Learning condition effect left vs right target first
evoked_dict= dict()
evoked_dict['Predicted/Left'] = evokedL_dict['Predicted/Left'] # Do right minus left?

evoked_dict['Predicted/Right'] = evokedL_dict['Predicted/Right']

evoked_dict['Random/Left'] = evokedL_dict['Random/Left']
evoked_dict['Random/Right'] = evokedL_dict['Random/Right']



pick = evoked_dict["Predicted/Left"].ch_names.index(channel2)

linestyles = dict(Left = '-', Right = '--')
colors = dict(Predicted = "Crimson", Random = "CornFlowerBlue")

xlim = (-0.2, 2.)

mne.viz.plot_compare_evokeds(evoked_dict, picks=pick, colors=colors,
                             linestyles=linestyles, split_legend=True)



In [ ]:
# Look at individual data - plot a whole strip at once 
subNum = '1'


#evoked_dummy = evokedO_dictAll['S' + str(subNum)]
linestyles = dict(Early = '-', Late = '--')
colors = dict(Predicted = "Crimson", Random = "CornFlowerBlue")

fig, ax = plt.subplots(nrows = 3, ncols = 1)

for channel in evoked_dict['Predicted/Early'].ch_names:
    

    #Plot specific conditions orienting
    evoked_dict= dict()
    evoked_dict['Predicted/Early'] = evokedO_dictAll['S' + str(subNum)]['Early/Predicted'] # Do right minus left?

    evoked_dict['Predicted/Late'] = evokedO_dictAll['S' + str(subNum)]['Late/Predicted']

    evoked_dict['Random/Early'] = evokedO_dictAll['S' + str(subNum)]['Early/Random']
    evoked_dict['Random/Late'] = evokedO_dictAll['S' + str(subNum)]['Late/Random']



    pick = evoked_dict["Predicted/Early"].ch_names.index(channel)

    

    xlim = (-0.2, 2.)

    mne.viz.plot_compare_evokeds(evoked_dict, picks=pick, colors=colors,
                                 linestyles=linestyles, split_legend=True, ax = ax)

In [ ]:
# Create dictionary of grandaverage objects per condition
GAVG_orienting = dict()


num = 0
for j in list(evokedO_dictAll['S01'].keys()):
    all_inst = []
    for i in subs:
        all_inst.append(evokedO_dictAll['S'+ str(i)][j])

    GAVG_orienting[j] =  mne.grand_average(all_inst, interpolate_bads=True, drop_bads=True)

    
GAVG_learning = dict()

'EEG pH1'
num = 0
for j in list(evokedL_dictAll['S18'].keys()):
    all_inst = []
    for i in subs:
        all_inst.append(evokedL_dictAll['S'+ str(i)][j])

    GAVG_learning[j] =  mne.grand_average(all_inst, interpolate_bads=True, drop_bads=True)

In [ ]:
#Plot specific conditions orienting
evoked_dict= dict()
evoked_dict['Predicted/Early'] = evokedO_dictAll['S1']['Early/Predicted'] # Do right minus left?

evoked_dict['Predicted/Late'] = evokedO_dictAll['S1']['Late/Predicted']

evoked_dict['Random/Early'] = evokedO_dictAll['S1']['Early/Random']
evoked_dict['Random/Late'] = evokedO_dictAll['S1']['Late/Random']



pick = evoked_dict["Predicted/Early"].ch_names.index('EEG pH1')

linestyles = dict(Early = '-', Late = '--')
colors = dict(Predicted = "Crimson", Random = "CornFlowerBlue")

xlim = (-0.2, 2.)

mne.viz.plot_compare_evokeds(evoked_dict, picks=pick, colors=colors,
                             linestyles=linestyles, split_legend=True)

In [ ]:
evokedO_dictAll

In [ ]:
# Run clustering algorithm

from mne.stats import permutation_cluster_test

channel = 'EEG pH1'
#chanInd = evokedO_dictAll['S18']['Early/Predicted'].ch_names.index(channel)

cond1 = 'Late/Predicted'
cond2 = 'Late/Random'

tWin = [0., 3.0]
timesOrig = evokedO_dictAll['S1'][cond1].times 
tInd = (timesOrig > tWin[0]) & (timesOrig< tWin[1])

subNum = 0

condition1 = np.zeros((len(subs),  sum(tInd)))
condition2 = np.zeros((len(subs),  sum(tInd)))

for i in subs:
    chanInd = evokedO_dictAll['S' + str(i)]['Early/Predicted'].ch_names.index(channel)
    
    condition1[subNum,:] = evokedO_dictAll['S' + str(i)][cond1].data[chanInd,tInd] # doesn't work for all elecs yet
    condition2[subNum,:] = evokedO_dictAll['S' + str(i)][cond2].data[chanInd,tInd]
    subNum += 1



#condition1 = condition1[:, chanInd, :]  # take only one channel to get a 2D array (subject x channels x time)
#condition2 = condition2[:, chanInd, :]  # take only one channel to get a 2D array

threshold = 2.0
T_obs, clusters, cluster_p_values, H0 = \
    permutation_cluster_test([condition1, condition2], n_permutations=1000,
                             threshold=threshold, tail=0, n_jobs=1)

In [ ]:
# Plot results
#times = GAVG_orienting['Early/Predicted'].times
times = timesOrig[tInd]
plt.close('all')
plt.subplot(211)
plt.title('Channel : ' + channel)
plt.plot(times, np.mean(condition1, axis = 0) - np.mean(condition2, axis = 0),
         label="ERP Contrast (Left - right response)")
plt.ylabel("EEG microV)")
plt.legend()
plt.subplot(212)
for i_c, c in enumerate(clusters):
    c = c[0]
    if cluster_p_values[i_c] <= 0.05:
        h = plt.axvspan(times[c.start], times[c.stop - 1],
                        color='r', alpha=0.3)
    else:
        plt.axvspan(times[c.start], times[c.stop - 1], color=(0.3, 0.3, 0.3),
                    alpha=0.3)
hf = plt.plot(times, T_obs, 'g')
plt.legend((h, ), ('cluster p-value < 0.05', ))
plt.xlabel("time (ms)")
plt.ylabel("f-values")
plt.show()

In [ ]:
#Plot specific conditions orienting
evoked_dict= dict()
evoked_dict['Predicted/Early/RespLeft'] = GAVG_orienting['Early/Predicted/RespLeft'] # Do right minus left?

evoked_dict['Predicted/Early/RespRight'] = GAVG_orienting['Early/Predicted/RespRight']

evoked_dict['Predicted/Late/RespLeft'] = GAVG_orienting['Late/Predicted/RespLeft']
evoked_dict['Predicted/Late/RespRight'] = GAVG_orienting['Late/Predicted/RespRight']



pick = evoked_dict["Predicted/Early/RespLeft"].ch_names.index('EEG pH1')

linestyles = dict(Early = '-', Late = '--')
colors = dict(RespLeft = "Coral", RespRight = "Green")

xlim = (-0.2, 2.)

mne.viz.plot_compare_evokeds(evoked_dict, picks=pick, colors=colors,
                             linestyles=linestyles, split_legend=True)

In [ ]:
#Plot specific conditions orienting
evoked_dict= dict()
evoked_dict['Random/Early/Left'] = GAVG_orienting['Early/Random/Left'] # Do right minus left?

evoked_dict['Random/Early/Right'] = GAVG_orienting['Early/Random/Right']

evoked_dict['Random/Late/Left'] = GAVG_orienting['Late/Random/Left']
evoked_dict['Random/Late/Right'] = GAVG_orienting['Late/Random/Right']



pick = evoked_dict["Random/Early/Left"].ch_names.index('PO8')

linestyles = dict(Early = '-', Late = '--')
colors = dict(Left = "Crimson", Right = "CornFlowerBlue")

xlim = (-0.2, 2.)

mne.viz.plot_compare_evokeds(evoked_dict, picks=pick, colors=colors,
                             linestyles=linestyles, split_legend=True)

## TFR analyses

In [ ]:
# Plot TFR
fig, axes = plt.subplots(nrows=3, ncols=2, figsize = (20,10))
fig.subplots_adjust(hspace=0.3)

ElecOI = 'EEG laH3'
fig.suptitle('Time-frequency for channel  ' + ' '.join(ElecOI))

PowOdict = evokedO_dict

times = evokedO_dict['Early/Predicted'].times
freqInd = frequencies > 1.0
tInd = (times > -1.5) & (times < 3.0) 

BLind = (times > -1.5) & (times < -1.3)

cmap = 'coolwarm'

vmax = 2.
vmin = -1*vmax


pick = PowOdict['Early/Predicted'].ch_names.index(ElecOI)

yLbl = np.round(frequencies, 1)
xLbl= times[tInd]
xticks = []
for i in xLbl: 
    if i%0.5 < 0.01:
        xticks.append(round(i,ndigits = 1 ))
    else: 
        xticks.append('')
blank = 0
yticks = []
for i in yLbl:
    if blank:
        yticks.append('')
        blank = 0
    else:
        yticks.append(i)
        blank = 1
    

data = np.mean(PowOdict['Late/Predicted']._data[:,pick,:,:], axis = 0)
base = np.mean(data[:,BLind], axis = 1)
data = (data - base[:,None])/base[:,None]

dataLP = data.copy()

sb.heatmap(data = data[:, tInd], xticklabels = xticks,  yticklabels = yticks, ax = axes[0,1], cmap = cmap, vmin = vmin, vmax = vmax)# , x = xLbl, y = yLbl)
axes[0,1].invert_yaxis()
axes[0,1].set(title= 'Late Predicted')     

del data, base

data = np.mean(PowOdict['Early/Predicted']._data[:,pick,:,:], axis = 0)
base = np.mean(data[:,BLind], axis = 1)
data = (data - base[:,None])/base[:,None]

dataEP = data.copy()

        
sb.heatmap(data = data[:, tInd], xticklabels = xticks,  yticklabels = yticks, ax = axes[0,0], cmap = cmap, vmin = vmin, vmax = vmax)# , x = xLbl, y = yLbl)
axes[0,0].invert_yaxis()
axes[0,0].set(title= 'Early Predicted')    

del data, base

data = np.mean(PowOdict['Late/Random']._data[:,pick,:,:], axis = 0)
base = np.mean(data[:,BLind], axis = 1)
data = (data - base[:,None])/base[:,None]

dataLR = data.copy()

        
sb.heatmap(data = data[:, tInd], xticklabels = xticks,  yticklabels = yticks , ax = axes[1,1],cmap = cmap, vmin = vmin, vmax = vmax)# , x = xLbl, y = yLbl)
axes[1,1].invert_yaxis()
axes[1,1].set(title= 'Late Random')        

del data, base
data = np.mean(PowOdict['Early/Random']._data[:,pick,:,:], axis = 0)
base = np.mean(data[:,BLind], axis = 1)
data = (data - base[:,None])/base[:,None]

dataER = data.copy()
        
sb.heatmap(data = data[:, tInd], xticklabels = xticks,  yticklabels = yticks, ax = axes[1,0],cmap = cmap, vmin = vmin, vmax = vmax)# , x = xLbl, y = yLbl)
axes[1,0].invert_yaxis()
axes[1,0].set(title= 'Early Random')
del data, base

datadiff = dataEP - dataER

sb.heatmap(data = datadiff[:, tInd], xticklabels = xticks,  yticklabels = yticks, ax = axes[2,0],cmap = cmap, vmin = vmin, vmax = vmax)# , x = xLbl, y = yLbl)
axes[2,0].invert_yaxis()
axes[2,0].set(title= 'Early Diff plot: Predicted - Random')

datadiff = dataLP - dataLR

sb.heatmap(data = datadiff[:, tInd], xticklabels = xticks,  yticklabels = yticks, ax = axes[2,1],cmap = cmap, vmin = vmin, vmax = vmax)# , x = xLbl, y = yLbl)
axes[2,1].invert_yaxis()
axes[2,1].set(title= 'Late Diff plot: Predicted - Random')

In [ ]:
# Plot TFR -- LEARNING
fig, axes = plt.subplots(nrows=3, ncols=1, figsize = (20,10))
fig.subplots_adjust(hspace=0.3)

#ElecOI = 'EEG rpH3'
fig.suptitle('Time-frequency for channel  ' + ' '.join(ElecOI))

PowOdict = evokedL_dict

times = evokedL_dict['Predicted'].times
freqInd = frequencies > 1.0
tInd = (times > -1.5) & (times < 12.0) 

BLind = (times > -1.5) & (times < -1.3)

cmap = 'coolwarm'

vmax = 2.
vmin = -1*vmax


pick = PowOdict['Predicted'].ch_names.index(ElecOI)

yLbl = np.round(frequencies, 1)
xLbl= times[tInd]
xticks = []
for i in xLbl: 
    if i%0.5 < 0.01:
        xticks.append(round(i,ndigits = 1 ))
    else: 
        xticks.append('')
blank = 0
yticks = []
for i in yLbl:
    if blank:
        yticks.append('')
        blank = 0
    else:
        yticks.append(i)
        blank = 1
    

data = np.mean(PowOdict['Predicted']._data[:,pick,:,:], axis = 0)
base = np.mean(data[:,BLind], axis = 1)
data = (data - base[:,None])/base[:,None]

dataP = data.copy()

sb.heatmap(data = data[:, tInd], xticklabels = xticks,  yticklabels = yticks, ax = axes[0], cmap = cmap, vmin = vmin, vmax = vmax)# , x = xLbl, y = yLbl)
axes[0].invert_yaxis()
axes[0].set(title= 'Predicted')     



   

del data, base

data = np.mean(PowOdict['Random']._data[:,pick,:,:], axis = 0)
base = np.mean(data[:,BLind], axis = 1)
data = (data - base[:,None])/base[:,None]

dataR = data.copy()

        
sb.heatmap(data = data[:, tInd], xticklabels = xticks,  yticklabels = yticks , ax = axes[1],cmap = cmap, vmin = vmin, vmax = vmax)# , x = xLbl, y = yLbl)
axes[1].invert_yaxis()
axes[1].set(title= 'Random')        

del data, base


datadiff = dataP - dataR

sb.heatmap(data = datadiff[:, tInd], xticklabels = xticks,  yticklabels = yticks, ax = axes[2],cmap = cmap, vmin = vmin, vmax = vmax)# , x = xLbl, y = yLbl)
axes[2].invert_yaxis()
axes[2].set(title= ' Diff plot: Predicted - Random')


In [ ]:
#Plot specific conditions orienting
evoked_dict= dict()
evoked_dict['Predicted/Early/Left'] = GAVG_orienting['Early/Predicted/Left'] # Do right minus left?

evoked_dict['Predicted/Early/Right'] = GAVG_orienting['Early/Predicted/Right']

evoked_dict['Predicted/Late/Left'] = GAVG_orienting['Late/Predicted/Left']
evoked_dict['Predicted/Late/Right'] = GAVG_orienting['Late/Predicted/Right']



pick = evoked_dict["Predicted/Early/Left"].ch_names.index('PO8')

linestyles = dict(Early = '-', Late = '--')
colors = dict(Left = "Crimson", Right = "CornFlowerBlue")

xlim = (-0.2, 2.)

mne.viz.plot_compare_evokeds(evoked_dict, picks=pick, colors=colors,
                             linestyles=linestyles, split_legend=True)

In [ ]:
# Run clustering algorithm

from mne.stats import permutation_cluster_test

channel = 'PO8' 
#chanInd = evokedO_dictAll['S18']['Early/Predicted'].ch_names.index(channel)

cond1 = 'Late/Predicted/Left'
cond2 = 'Late/Predicted/Right'

tWin = [0., 3.0]
timesOrig = evokedO_dictAll['S18'][cond1].times 
tInd = (timesOrig > tWin[0]) & (timesOrig< tWin[1])

subNum = 0

condition1 = np.zeros((len(subs),  sum(tInd)))
condition2 = np.zeros((len(subs),  sum(tInd)))

for i in subs:
    chanInd = evokedO_dictAll['S' + str(i)]['Early/Predicted'].ch_names.index(channel)
    
    condition1[subNum,:] = evokedO_dictAll['S' + str(i)][cond1].data[chanInd,tInd] # doesn't work for all elecs yet
    condition2[subNum,:] = evokedO_dictAll['S' + str(i)][cond2].data[chanInd,tInd]
    subNum += 1



#condition1 = condition1[:, chanInd, :]  # take only one channel to get a 2D array (subject x channels x time)
#condition2 = condition2[:, chanInd, :]  # take only one channel to get a 2D array

threshold = 2.0
T_obs, clusters, cluster_p_values, H0 = \
    permutation_cluster_test([condition1, condition2], n_permutations=1000,
                             threshold=threshold, tail=0, n_jobs=1)

In [ ]:
# Run clustering algorithm

from mne.stats import permutation_cluster_test

channel = 'C3' 
#chanInd = evokedO_dictAll['S18']['Early/Predicted'].ch_names.index(channel)

cond1 = 'Late/Predicted/RespLeft'
cond2 = 'Late/Predicted/RespRight'

tWin = [0., 3.0]
timesOrig = evokedO_dictAll['S18'][cond1].times 
tInd = (timesOrig > tWin[0]) & (timesOrig< tWin[1])

subNum = 0

condition1 = np.zeros((len(subs),  sum(tInd)))
condition2 = np.zeros((len(subs),  sum(tInd)))

for i in subs:
    chanInd = evokedO_dictAll['S' + str(i)]['Early/Predicted'].ch_names.index(channel)
    
    condition1[subNum,:] = evokedO_dictAll['S' + str(i)][cond1].data[chanInd,tInd] # doesn't work for all elecs yet
    condition2[subNum,:] = evokedO_dictAll['S' + str(i)][cond2].data[chanInd,tInd]
    subNum += 1



#condition1 = condition1[:, chanInd, :]  # take only one channel to get a 2D array (subject x channels x time)
#condition2 = condition2[:, chanInd, :]  # take only one channel to get a 2D array

threshold = 2.0
T_obs, clusters, cluster_p_values, H0 = \
    permutation_cluster_test([condition1, condition2], n_permutations=1000,
                             threshold=threshold, tail=0, n_jobs=1)

In [ ]:
# Plot results
#times = GAVG_orienting['Early/Predicted'].times
times = timesOrig[tInd]
plt.close('all')
plt.subplot(211)
plt.title('Channel : ' + channel)
plt.plot(times, np.mean(condition1, axis = 0) - np.mean(condition2, axis = 0),
         label="ERP Contrast (Predicted - Random)")
plt.ylabel("EEG microV)")
plt.legend()
plt.subplot(212)
for i_c, c in enumerate(clusters):
    c = c[0]
    if cluster_p_values[i_c] <= 0.05:
        h = plt.axvspan(times[c.start], times[c.stop - 1],
                        color='r', alpha=0.3)
    else:
        plt.axvspan(times[c.start], times[c.stop - 1], color=(0.3, 0.3, 0.3),
                    alpha=0.3)
hf = plt.plot(times, T_obs, 'g')
#plt.legend((h, ), ('cluster p-value < 0.05', ))
plt.xlabel("time (ms)")
plt.ylabel("f-values")
plt.show()

In [ ]:
# Run clustering algorithm Learning

from mne.stats import permutation_cluster_test

channel = 'PO7' 
#chanInd = evokedO_dictAll['S18']['Early/Predicted'].ch_names.index(channel)

cond1 = 'Random/Left'
cond2 = 'Random/Right'

tWin = [0., 3.0]
timesOrig = evokedL_dictAll['S18'][cond1].times 
tInd = (timesOrig > tWin[0]) & (timesOrig< tWin[1])

subNum = 0

condition1 = np.zeros((len(subs),  sum(tInd)))
condition2 = np.zeros((len(subs),  sum(tInd)))

for i in subs:
    chanInd = evokedL_dictAll['S' + str(i)]['Predicted/Left'].ch_names.index(channel)
    
    condition1[subNum,:] = evokedL_dictAll['S' + str(i)][cond1].data[chanInd,tInd] # doesn't work for all elecs yet
    condition2[subNum,:] = evokedL_dictAll['S' + str(i)][cond2].data[chanInd,tInd]
    subNum += 1



#condition1 = condition1[:, chanInd, :]  # take only one channel to get a 2D array (subject x channels x time)
#condition2 = condition2[:, chanInd, :]  # take only one channel to get a 2D array

threshold = 2.0
T_obs, clusters, cluster_p_values, H0 = \
    permutation_cluster_test([condition1, condition2], n_permutations=1000,
                             threshold=threshold, tail=0, n_jobs=1)

In [ ]:
# Plot Learning condition effect left vs right target first
evoked_dict= dict()
evoked_dict['Predicted'] = GAVG_learning['Predicted'] 

evoked_dict['Predicted'] = GAVG_learning['Predicted']

evoked_dict['Random'] = GAVG_learning['Random']
evoked_dict['Random'] = GAVG_learning['Random']



pick = evoked_dict["Predicted"].ch_names.index('PZ')

linestyles = dict(Left = '-', Right = '--')
colors = dict(Predicted = "Crimson", Random = "CornFlowerBlue")

xlim = (-0.2, 2.)

mne.viz.plot_compare_evokeds(evoked_dict, picks=pick, colors=colors,
                             linestyles=linestyles, split_legend=True)

In [ ]:
# Plot Learning condition effect left vs right target first
evoked_dict= dict()
evoked_dict['Predicted/Left'] = evokedL_dictAll['S' + str(subNum)]['Predicted/Left'] # Do right minus left?

evoked_dict['Predicted/Right'] = evokedL_dictAll['S' + str(subNum)]['Predicted/Right']

evoked_dict['Random/Left'] = evokedL_dictAll['S' + str(subNum)]['Random/Left']
evoked_dict['Random/Right'] = evokedL_dictAll['S' + str(subNum)]['Random/Right']



pick = evoked_dict["Predicted/Left"].ch_names.index('PO8')

linestyles = dict(Left = '-', Right = '--')
colors = dict(Predicted = "Crimson", Random = "CornFlowerBlue")

xlim = (-0.2, 2.)

mne.viz.plot_compare_evokeds(evoked_dict, picks=pick, colors=colors,
                             linestyles=linestyles, split_legend=True)

In [ ]:
GAVG_learning

In [ ]:
# Plot grandaverage

times = [1.1, 1.2, 1.3, 1.4, 1.5]
condition = 'Early/Random'
GAVG_orienting[condition].plot_topomap(times = times, ch_type = 'eeg', time_unit = 's')
condition = 'Early/Predicted'
GAVG_orienting[condition].plot_topomap(times = times, ch_type = 'eeg', time_unit = 's')

times = [2.3, 2.4, 2.5, 2.6, 2.7]
condition = 'Late/Random'
GAVG_orienting[condition].plot_topomap(times = times, ch_type = 'eeg', time_unit = 's')
condition = 'Late/Predicted'
GAVG_orienting[condition].plot_topomap(times = times, ch_type = 'eeg', time_unit = 's')

In [ ]:
# crop/trim

evoked_dict= dict()
evoked_dict['Predicted/Early'] = GAVG_orienting['Early/Predicted']

evoked_dict['Predicted/Late'] = GAVG_orienting['Late/Predicted']

evoked_dict['Random/Early'] = GAVG_orienting['Early/Random']
evoked_dict['Random/Late'] = GAVG_orienting['Late/Random']

pick = evoked_dict["Predicted/Early"].ch_names.index('PZ')

linestyles = dict(Early = '-', Late = '--')
colors = dict(Predicted = "Crimson", Random = "CornFlowerBlue")

xlim = (-0.2, 2.)

fig = mne.viz.plot_compare_evokeds(evoked_dict, picks=pick, colors=colors,
                             linestyles=linestyles, split_legend=True)

fig.set_figheight(8)
fig.set_figwidth(20)

In [ ]:
mne.viz.plot_compare_evokeds?

In [ ]:
xlim = (1.5, 2.)
evoked_dict['Predicted/Early'].plot_image(xlim = xlim)
evoked_dict['Random/Early'].plot_image(xlim = xlim)
evoked_dict['Predicted/Late'].plot_image(xlim = xlim)
evoked_dict['Random/Late'].plot_image(xlim = xlim)

In [ ]:
evoked_dict['Predicted/Early'].plot_joint()
evoked_dict['Random/Early'].plot_joint()
evoked_dict['Random/Late'].plot_joint()

In [ ]:
mne.viz.plot_evoked_topo(evoked_dict['Predicted/Late'],background_color='w')

In [ ]:
mne.viz.plot_compare_evokeds?

In [ ]:
evoked_dict= dict()
evoked_dict['Predicted/Left'] = GAVG_learning['Predicted/Left']
evoked_dict['Predicted/Right'] = GAVG_learning['Predicted/Right']
evoked_dict['Random/Left'] = GAVG_learning['Random/Left']
evoked_dict['Random/Right'] = GAVG_learning['Random/Right']

pick = evoked_dict["Predicted/Left"].ch_names.index('PZ')
linestyles = dict(Left = '-', Right = '--')
colors = dict(Predicted = "Crimson", Random = "CornFlowerBlue")

mne.viz.plot_compare_evokeds(evoked_dict, picks=pick, colors=colors,
                             linestyles=linestyles, split_legend=True)

In [ ]:
mne.viz.plot_compare_evokeds?

In [ ]:
%matplotlib notebook
evoked_dict= dict()
evoked_dict['nRep1'] = GAVG_learning['nRep1']
evoked_dict['nRep2'] = GAVG_learning['nRep2']
evoked_dict['nRep3'] = GAVG_learning['nRep3']
evoked_dict['nRep4'] = GAVG_learning['nRep4']
evoked_dict['nRep5'] = GAVG_learning['nRep5']
evoked_dict['nRep6'] = GAVG_learning['nRep6']

#pick = evoked_dict["nRep1"].ch_names.index('PZ')

pick = []
elec = ['EEG pH1']
for k in elec: 
    pick.append(evoked_dict['nRep1'].ch_names.index(k))
#linestyles = dict(Left = '-', Right = '--')
#colors = dict(nRep1 = 'Gainsboro', nRep2 = 'LightGray', nRep3 = 'Silver', nRep4 = 'DarkGray', nRep5 = 'Gray', nRep6 = 'DimGray' )
cmap = plt.cm.get_cmap('gray')
#colors = dict(nRep1 = cmap(0.2), nRep2 = cmap(0.3), nRep3 = cmap(0.4), nRep4 = cmap(0.5), nRep5 = cmap(0.6), nRep6 = cmap(0.7) )
colors = dict(nRep1 = cmap(0.7), nRep2 = cmap(0.6), nRep3 = cmap(0.5), nRep4 = cmap(0.4))#, nRep5 = cmap(0.3), nRep6 = cmap(0.2) )



mne.viz.plot_compare_evokeds(evoked_dict, picks=pick, colors = colors, split_legend=True, vlines = [-1.2, 0, 0.8, 2.0])

In [ ]:
evoked_dict= dict()
evoked_dict['Predicted/nRep1'] = evokedL_dict['Predicted/nRep1']
evoked_dict['Predicted/nRep2'] = evokedL_dict['Predicted/nRep2']
evoked_dict['Predicted/nRep3'] = evokedL_dict['Predicted/nRep3']
evoked_dict['Predicted/nRep4'] = evokedL_dict['Predicted/nRep4']
evoked_dict['Predicted/nRep5'] = evokedL_dict['Predicted/nRep5']
evoked_dict['Predicted/nRep6'] = evokedL_dict['Predicted/nRep6']

pick = evoked_dict["Predicted/nRep1"].ch_names.index('EEG aH1')
#linestyles = dict(Left = '-', Right = '--')
colors = dict(nRep1 = 'Gainsboro', nRep2 = 'LightGray', nRep3 = 'Silver', nRep4 = 'DarkGray', nRep5 = 'Gray', nRep6 = 'DimGray' )

mne.viz.plot_compare_evokeds(evoked_dict, picks=pick, colors = colors, split_legend=True)

In [ ]:
evokedL_dict

In [ ]:
evoked_dict= dict()
evoked_dict['Random/nRep1'] = GAVG_learning['Random/nRep1']
evoked_dict['Random/nRep2'] = GAVG_learning['Random/nRep2']
evoked_dict['Random/nRep3'] = GAVG_learning['Random/nRep3']
evoked_dict['Random/nRep4'] = GAVG_learning['Random/nRep4']
evoked_dict['Random/nRep5'] = GAVG_learning['Random/nRep5']
evoked_dict['Random/nRep6'] = GAVG_learning['Random/nRep6']

pick = evoked_dict["Random/nRep1"].ch_names.index('POZ')

pick = []
elec = ['POZ']#'PZ', 'CPZ', 'POZ', 'P1']
for k in elec: 
    pick.append(evoked_dict['Random/nRep1'].ch_names.index(k))
#linestyles = dict(Left = '-', Right = '--')
colors = dict(nRep1 = 'Gainsboro', nRep2 = 'LightGray', nRep3 = 'Silver', nRep4 = 'DarkGray', nRep5 = 'Gray', nRep6 = 'DimGray' )
cmap = plt.cm.get_cmap('Greens')
#colors = dict(nRep1 = cmap(0.6), nRep2 = cmap(0.5), nRep3 = cmap(0.4), nRep4 = cmap(0.3), nRep5 = cmap(0.2), nRep6 = cmap(0.1) )


mne.viz.plot_compare_evokeds(evoked_dict, picks=pick, colors = colors, split_legend=True)

In [ ]:
cmap(0.0)

In [ ]:
# N2PC - look at 200-300ms post stim, PO7/PO8

evoked_dict= dict()
evoked_dict['Predicted/Early/Left'] = GAVG_orienting['Early/Predicted/Left'] # Do right minus left?

evoked_dict['Predicted/Late/Left'] = GAVG_orienting['Late/Predicted/Left']

evoked_dict['Random/Early/Left'] = GAVG_orienting['Early/Random/Left']
evoked_dict['Random/Late/Left'] = GAVG_orienting['Late/Random/Left']



pick = evoked_dict["Predicted/Early/Left"].ch_names.index('PO8')

linestyles = dict(Early = '-', Late = '--')
colors = dict(Predicted = "Crimson", Random = "CornFlowerBlue")

xlim = (-0.2, 2.)

mne.viz.plot_compare_evokeds(evoked_dict, picks=pick, colors=colors,
                             linestyles=linestyles, split_legend=True)



In [ ]:
# N2PC - look at 200-300ms post stim, PO7/PO8. Right target: do PO7-PO8 (contra - ipsi) Left: PO8-PO7

evoked_dict= dict()
evoked_dict['Predicted/Early/Right'] = GAVG_orienting['Early/Predicted/Right'] # Do right minus left?

evoked_dict['Predicted/Late/Right'] = GAVG_orienting['Late/Predicted/Right']

evoked_dict['Random/Early/Right'] = GAVG_orienting['Early/Random/Right']
evoked_dict['Random/Late/Right'] = GAVG_orienting['Late/Random/Right']



pick = evoked_dict["Predicted/Early/Right"].ch_names.index('PZ')

linestyles = dict(Early = '-', Late = '--')
colors = dict(Predicted = "Crimson", Random = "CornFlowerBlue")

xlim = (-0.2, 2.)

mne.viz.plot_compare_evokeds(evoked_dict, picks=pick, colors=colors,
                             linestyles=linestyles, split_legend=True)


In [ ]:
evokedO_dictAll

In [ ]:
# Calculate difference waves for n2pc

leftElec = 'PO7'
rightElec = 'PO8'
#LRdict = dict()
#LRdictAll = dict()

emptyNp = np.zeros((len(subs), evokedO_dictAll['S18']['Early/Random/Left'].data.shape[1]))

LRdict = {'PO8/Random/Early': emptyNp.copy(), 'PO7/Random/Early': emptyNp.copy(), 'PO7/Predicted/Early': emptyNp.copy(), 'PO8/Predicted/Early': emptyNp.copy(),
      'PO8/Random/Late': emptyNp.copy(), 'PO7/Random/Late': emptyNp.copy(),  'PO7/Predicted/Late': emptyNp.copy(), 'PO8/Predicted/Late': emptyNp.copy()}
num = 0
for i in subs:
    
    
    leftInd = evokedO_dictAll['S' + str(i)]['Early/Random/Left'].ch_names.index(leftElec)
    rightInd = evokedO_dictAll['S' + str(i)]['Early/Random/Left'].ch_names.index(rightElec)
    
    # Contra - ipsi, for left electrode and right electrode
    #LRdict['PO8/Random/Early'][num,:] = evokedO_dictAll['S' + str(i)]['Early/Random/Right'].data[rightInd,:] - evokedO_dictAll['S' + str(i)]['Early/Random/Left'].data[rightInd,:]
    #LRdict['PO7/Random/Early'][num,:] = evokedO_dictAll['S' + str(i)]['Early/Random/Left'].data[leftInd,:] - evokedO_dictAll['S' + str(i)]['Early/Random/Right'].data[leftInd,:]

    LRdict['PO7/Predicted/Early'][num,:]= evokedO_dictAll['S' + str(i)]['Early/Predicted/Right'].data[rightInd,:] - evokedO_dictAll['S' + str(i)]['Early/Predicted/Left'].data[rightInd,:]
    LRdict['PO8/Predicted/Early'][num,:]= evokedO_dictAll['S' + str(i)]['Early/Predicted/Left'].data[leftInd,:] - evokedO_dictAll['S' + str(i)]['Early/Predicted/Right'].data[leftInd,:]

    #LRdict['PO8/Random/Late'][num,:]= evokedO_dictAll['S' + str(i)]['Late/Random/Right'].data[rightInd,:] - evokedO_dictAll['S' + str(i)]['Late/Random/Left'].data[rightInd,:]
    #LRdict['PO7/Random/Late'][num,:] = evokedO_dictAll['S' + str(i)]['Late/Random/Left'].data[leftInd,:] - evokedO_dictAll['S' + str(i)]['Late/Random/Right'].data[leftInd,:]

    LRdict['PO7/Predicted/Late'][num,:]= evokedO_dictAll['S' + str(i)]['Late/Predicted/Right'].data[rightInd,:] - evokedO_dictAll['S' + str(i)]['Late/Predicted/Left'].data[rightInd,:]
    LRdict['PO8/Predicted/Late'][num,:] = evokedO_dictAll['S' + str(i)]['Late/Predicted/Left'].data[leftInd,:] - evokedO_dictAll['S' + str(i)]['Late/Predicted/Right'].data[leftInd,:]

    #LRdictAll['S' + str(i)] = LRdict
    num += 1

  
    

In [ ]:
# left vs right responses 


leftElec = 'C3'
rightElec = 'C4'
#LRdict = dict()
#LRdictAll = dict()

emptyNp = np.zeros((len(subs), evokedO_dictAll['S18']['Early/Random/Left'].data.shape[1]))

LRrespdict = {'C4/Random/Early': emptyNp.copy(), 'C3/Random/Early': emptyNp.copy(), 'C3/Predicted/Early': emptyNp.copy(), 'C4/Predicted/Early': emptyNp.copy(),
      'C4/Random/Late': emptyNp.copy(), 'C3/Random/Late': emptyNp.copy(),  'C3/Predicted/Late': emptyNp.copy(), 'C4/Predicted/Late': emptyNp.copy()}
num = 0
for i in subs:
    
    
    leftInd = evokedO_dictAll['S' + str(i)]['Early/Random/Left'].ch_names.index(leftElec)
    rightInd = evokedO_dictAll['S' + str(i)]['Early/Random/Left'].ch_names.index(rightElec)
    
    # Contra - ipsi, for left electrode and right electrode
    LRrespdict['C4/Random/Early'][num,:] = evokedO_dictAll['S' + str(i)]['Early/Random/RespLeft'].data[rightInd,:] - evokedO_dictAll['S' + str(i)]['Early/Random/RespRight'].data[rightInd,:]
    LRrespdict['C3/Random/Early'][num,:] = evokedO_dictAll['S' + str(i)]['Early/Random/RespRight'].data[leftInd,:] - evokedO_dictAll['S' + str(i)]['Early/Random/RespLeft'].data[leftInd,:]

    LRrespdict['C4/Predicted/Early'][num,:]= evokedO_dictAll['S' + str(i)]['Early/Predicted/RespLeft'].data[rightInd,:] - evokedO_dictAll['S' + str(i)]['Early/Predicted/RespRight'].data[rightInd,:]
    LRrespdict['C3/Predicted/Early'][num,:]= evokedO_dictAll['S' + str(i)]['Early/Predicted/RespRight'].data[leftInd,:] - evokedO_dictAll['S' + str(i)]['Early/Predicted/RespLeft'].data[leftInd,:]

    LRrespdict['C3/Random/Late'][num,:]= evokedO_dictAll['S' + str(i)]['Late/Random/RespRight'].data[rightInd,:] - evokedO_dictAll['S' + str(i)]['Late/Random/RespLeft'].data[rightInd,:]
    LRrespdict['C4/Random/Late'][num,:] = evokedO_dictAll['S' + str(i)]['Late/Random/RespLeft'].data[leftInd,:] - evokedO_dictAll['S' + str(i)]['Late/Random/RespRight'].data[leftInd,:]

    LRrespdict['C3/Predicted/Late'][num,:]= evokedO_dictAll['S' + str(i)]['Late/Predicted/RespRight'].data[rightInd,:] - evokedO_dictAll['S' + str(i)]['Late/Predicted/RespLeft'].data[rightInd,:]
    LRrespdict['C4/Predicted/Late'][num,:] = evokedO_dictAll['S' + str(i)]['Late/Predicted/RespLeft'].data[leftInd,:] - evokedO_dictAll['S' + str(i)]['Late/Predicted/RespRight'].data[leftInd,:]

    #LRdictAll['S' + str(i)] = LRdict
    num += 1



In [ ]:
import seaborn as sb

%matplotlib notebook 

time = np.squeeze(np.arange(-3. - 1./250., 4., 1./250.))#evokedO_dictAll['S18']['Early/Random'].info
linestyles = dict(Early = '-', Late = '--')
colors = dict(Predicted = "Crimson", Random = "CornFlowerBlue")

timewin = [0., 3.]
timeInd = (time > timewin[0]) * (time< timewin[1])
fig1, ax1 = plt.subplots()
ax1.set_title('C4 contra - ipsi')
# Plot PO8
for i in list(LRrespdict.keys()):
    if 'C4' in i:
        print(time.shape)
        print(LRrespdict[i].shape)
        #print(LRdict[i].transpose().shape)
        if 'Predicted' in i:
            color = 'Crimson'
        elif 'Random' in i:
            color = 'CornFlowerBlue'
        
        if 'Early' in i:
            linestyle = '-'
        elif 'Late' in i:
            linestyle = '--'
        
        sb.tsplot(time = time[timeInd], data = LRrespdict[i][:,timeInd], color = color, linestyle = linestyle, ax = ax1, ci = 0)# , colors = colors, linestyles = linestyles)
        

        
fig2, ax2 = plt.subplots()

ax2.set_title('C3 contra - ipsi')

 # Plot PO7
for i in list(LRrespdict.keys()):
    if 'C3' in i:
        print(time.shape)
        print(LRrespdict[i].shape)
        #print(LRdict[i].transpose().shape)
        if 'Predicted' in i:
            color = 'Crimson'
        elif 'Random' in i:
            color = 'CornFlowerBlue'
        
        if 'Early' in i:
            linestyle = '-'
        elif 'Late' in i:
            linestyle = '--'
        
        sb.tsplot(time = time[timeInd], data = LRrespdict[i][:,timeInd], color = color, linestyle = linestyle, ax = ax2, ci = 0)#

In [ ]:
import seaborn as sb

%matplotlib notebook 

time = np.squeeze(np.arange(-3. - 1./250., 4., 1./250.))#evokedO_dictAll['S18']['Early/Random'].info
linestyles = dict(Early = '-', Late = '--')
colors = dict(Predicted = "Crimson", Random = "CornFlowerBlue")

timewin = [0., 2.4]
timeInd = (time > timewin[0]) * (time< timewin[1])
fig1, ax1 = plt.subplots()
ax1.set_title('PO8 contra - ipsi')
# Plot PO8
for i in list(LRdict.keys()):
    if 'PO8' in i:
        print(time.shape)
        print(LRdict[i].shape)
        #print(LRdict[i].transpose().shape)
        if 'Predicted' in i:
            color = 'Crimson'
        elif 'Random' in i:
            color = 'CornFlowerBlue'
        
        if 'Early' in i:
            linestyle = '-'
        elif 'Late' in i:
            linestyle = '--'
        
        sb.tsplot(time = time[timeInd], data = LRdict[i][:,timeInd], color = color, linestyle = linestyle, ax = ax1)# , colors = colors, linestyles = linestyles)
        

        
fig2, ax2 = plt.subplots()

ax2.set_title('PO7 contra - ipsi')

 # Plot PO7
for i in list(LRdict.keys()):
    if 'PO7' in i:
        print(time.shape)
        print(LRdict[i].shape)
        #print(LRdict[i].transpose().shape)
        if 'Predicted' in i:
            color = 'Crimson'
        elif 'Random' in i:
            color = 'CornFlowerBlue'
        
        if 'Early' in i:
            linestyle = '-'
        elif 'Late' in i:
            linestyle = '--'
        
        sb.tsplot(time = time[timeInd], data = LRdict[i][:,timeInd], color = color, linestyle = linestyle, ax = ax2)#

In [ ]:
# Plot PO7
for i in list(LRdict.keys()):
    if 'PO7' in i:
        print(time.shape)
        print(LRdict[i].shape)
        #print(LRdict[i].transpose().shape)
        if 'Predicted' in i:
            color = 'Crimson'
        elif 'Random' in i:
            color = 'CornFlowerBlue'
        
        if 'Early' in i:
            linestyle = '-'
        elif 'Late' in i:
            linestyle = '--'
        
        sb.tsplot(time = time[timeInd], data = LRdict[i][:,timeInd], color = color, linestyle = linestyle)#

In [ ]:
evokedO_dictAll['S18']['Early/Random/Left'].data.shape

In [ ]:
sb.tsplot?

In [ ]:
LRdict[i][0,:]

In [ ]:
evoked_dict= dict()
evoked_dict['Predicted/Early'] = GAVG_orienting['Early/Predicted']

evoked_dict['Predicted/Late'] = GAVG_orienting['Late/Predicted']

evoked_dict['Random/Early'] = GAVG_orienting['Early/Random']
evoked_dict['Random/Late'] = GAVG_orienting['Late/Random']

pick = evoked_dict["Predicted/Early"].ch_names.index('PZ')

linestyles = dict(Early = '-', Late = '--')
colors = dict(Predicted = "Crimson", Random = "CornFlowerBlue")

mne.viz.plot_compare_evokeds(evoked_dict, picks=pick, colors=colors,
                             linestyles=linestyles, split_legend=True)

In [ ]:
#evokedL_dictAll

P300amp = dict()
P300lat = dict()
elec = 'PZ'
srate = 250

latencies = [0.2, 0.5]
for i in subs:
    

    tmp = dict()
    tmp2 = dict()
    for j in list(evokedL_dictAll['S18'].keys()):
        
        elecInd = evokedL_dictAll['S' + str(i)][j].ch_names.index(elec)
        timeInd = (evokedL_dictAll['S' + str(i)][j].times > latencies[0])*(evokedL_dictAll['S' + str(i)][j].times < latencies[1])
        tmp[j] = np.max(evokedL_dictAll['S'+ str(i)][j].data[elecInd, timeInd])
        tmp2[j] = latencies[0] + np.argmax(evokedL_dictAll['S'+ str(i)][j].data[elecInd, timeInd])/srate
    
    P300amp['S' + str(i)] = tmp
    P300lat['S' + str(i)] = tmp2

In [ ]:
ev_dict = dict()
tmp_dict = dict()




In [ ]:
# average across several electrodes

P300amp = dict()
P300lat = dict()
elec = ['PZ', 'POZ', 'P1', 'P2', 'CPZ']
srate = 250

latencies = [1.0, 1.4] # targets at 0.8s and 2s
latencies2 = [2.2, 2.8]

nRep = []
condition = []
subject = []

amp = []
lats = []
amp2 = []
lats2 = []

for i in subs:
    

    tmp = dict()
    tmp2 = dict()
    for j in list(evokedL_dictAll['S18'].keys()):
        if 'nRep' in j:
            
            if 'Random' in j or 'Predicted' in j:
            
                subject.append('S' + str(i))

                if 'Random' in j:
                    condition.append('Random')
                elif 'Predicted' in j:
                    condition.append('Predicted')

                if '1' in j:
                    nRep.append(1.0)
                elif '2' in j:
                    nRep.append(2.0)
                elif '3' in j: 
                    nRep.append(3.0)
                elif '4' in j:
                    nRep.append(4.0)
                elif '5' in j: 
                    nRep.append(5.0)
                elif '6' in j:
                    nRep.append(6.0)

                elecIndTmp = []
                for k in elec: 
                    elecIndTmp.append(evokedL_dictAll['S' + str(i)][j].ch_names.index(k))

                timeInd = (evokedL_dictAll['S' + str(i)][j].times > latencies[0])*(evokedL_dictAll['S' + str(i)][j].times < latencies[1])

                dataLen = evokedL_dictAll['S'+ str(i)][j].data.shape[0]
                elecInd = np.zeros((dataLen,1))
                elecInd[elecIndTmp] = True
                elecInd = np.squeeze(elecInd.astype(bool))

                data1 = []
                data1_2 = []
                data2_2 = []

                data1 = evokedL_dictAll['S'+ str(i)][j].data[elecInd, :]
                data1_2 = data1[:,timeInd]

                amp.append( np.max(np.mean(data1_2, axis = 0)))
                lats.append(latencies[0] + np.argmax(np.mean(data1_2, axis = 0))/srate)

                timeInd2 = (evokedL_dictAll['S' + str(i)][j].times > latencies2[0])*(evokedL_dictAll['S' + str(i)][j].times < latencies2[1])


                data2_2 = data1[:,timeInd2]

                amp2.append( np.max(np.mean(data2_2, axis = 0)))
                lats2.append(latencies2[0] + np.argmax(np.mean(data2_2, axis= 0))/srate)
            
            
    
d = {'Subject': subject, 'Condition': condition, 'nRep': nRep, 'PeakLatency1': lats, 'PeakAmplitude1': amp, 'PeakLatency2': lats2, 'PeakAmplitude2': amp2}
df = pd.DataFrame(d)

In [ ]:
evokedL_dictAll['S' + str(i)][j].data[elecInd,timeInd]

In [ ]:
sum(timeInd)

In [ ]:
# single electrode only
P300amp = dict()
P300lat = dict()
elec = 'PZ'
srate = 250

latencies = [1.0, 1.4] # targets at 0.8s and 2s
latencies2 = [2.2, 2.8]

nRep = []
condition = []
subject = []

amp = []
lats = []
amp2 = []
lats2 = []

for i in subs:
    

    tmp = dict()
    tmp2 = dict()
    for j in list(evokedL_dictAll['S18'].keys()):
        if 'nRep' in j:
            
            subject.append('S' + str(i))
            
            if 'Random' in j:
                condition.append('Random')
            elif 'Predicted' in j:
                condition.append('Predicted')
            
            if '1' in j:
                nRep.append(1.0)
            elif '2' in j:
                nRep.append(2.0)
            elif '3' in j: 
                nRep.append(3.0)
            elif '4' in j:
                nRep.append(4.0)
            elif '5' in j: 
                nRep.append(5.0)
            elif '6' in j:
                nRep.append(6.0)
            
            elecInd = evokedL_dictAll['S' + str(i)][j].ch_names.index(elec)
            timeInd = (evokedL_dictAll['S' + str(i)][j].times > latencies[0])*(evokedL_dictAll['S' + str(i)][j].times < latencies[1])
        
            amp.append( np.max(evokedL_dictAll['S'+ str(i)][j].data[elecInd, timeInd]))
            lats.append(latencies[0] + np.argmax(evokedL_dictAll['S'+ str(i)][j].data[elecInd, timeInd])/srate)
            
            timeInd2 = (evokedL_dictAll['S' + str(i)][j].times > latencies2[0])*(evokedL_dictAll['S' + str(i)][j].times < latencies2[1])
        
            amp2.append( np.max(evokedL_dictAll['S'+ str(i)][j].data[elecInd, timeInd2]))
            lats2.append(latencies2[0] + np.argmax(evokedL_dictAll['S'+ str(i)][j].data[elecInd, timeInd2])/srate)
            
            
    
d = {'Subject': subject, 'Condition': condition, 'nRep': nRep, 'PeakLatency1': lats, 'PeakAmplitude1': amp, 'PeakLatency2': lats2, 'PeakAmplitude2': amp2}
df = pd.DataFrame(d)

In [ ]:
print(len(subject))
print(len(condition))
print(len(nRep))



In [ ]:
df

In [ ]:
import seaborn as sb
sb.catplot(x = 'nRep', y = 'PeakAmplitude1', data = df, hue='Condition', kind = 'point')

sb.catplot(x = 'nRep', y = 'PeakAmplitude2', data = df, hue = 'Condition', kind = 'point' )


sb.catplot(x = 'nRep', y = 'PeakLatency1', data = df, hue = 'Condition', kind = 'point' )

sb.catplot(x = 'nRep', y = 'PeakLatency2', data = df, hue = 'Condition', kind = 'point' )

In [ ]:
sb.catplot?

In [ ]:
import seaborn as sb
import 

In [ ]:
import statsmodels.formula.api as smf

#dfLME = dfO.dropna(subset = ['pixelError']) # do 'RTselec1' 'MoveOnset' or 'pixelError'
dfLME = df.copy()
#dfLME['Scene'] = dfLME['Scene'].astype(str)
#dfLME['Delay'] = dfLME['Delay'].astype(str)
#dfLME= dfLME[dfLME['MoveOnset']> 100]


modelLMERT = smf.mixedlm("PeakAmplitude1 ~Condition + nRep + nRep*Condition",dfLME, groups = dfLME['Subject'])
LMERTFit = modelLMERT.fit()
print(LMERTFit.summary())

In [ ]:
# single electrode only
P300amp = dict()
P300lat = dict()
elec = 'PZ'
srate = 250

latencies = [1.0, 1.4] # targets at 0.8s and 2s
latencies2 = [2.2, 2.8]

nRep = []
condition = []
subject = []
delay = []

amp = []
lats = []
amp2 = []
lats2 = []

for i in subs:
    

    tmp = dict()
    tmp2 = dict()
    for j in list(['Early/Random', 'Early/Predicted', 'Late/Random', 'Late/Predicted']):
        if 'Early' in j:
            delay.append('Early')
            subject.append('S' + str(i))
            
            if 'Random' in j:
                condition.append('Random')
            elif 'Predicted' in j:
                condition.append('Predicted')
            
            
            
            elecInd = evokedO_dictAll['S' + str(i)][j].ch_names.index(elec)
            timeInd = (evokedO_dictAll['S' + str(i)][j].times > latencies[0])*(evokedO_dictAll['S' + str(i)][j].times < latencies[1])
        
            amp.append( np.max(evokedO_dictAll['S'+ str(i)][j].data[elecInd, timeInd]))
            lats.append(latencies[0] + np.argmax(evokedO_dictAll['S'+ str(i)][j].data[elecInd, timeInd])/srate - 0.8)
            
        elif 'Late' in j:
            delay.append('Late')
            subject.append('S' + str(i))
            
            if 'Random' in j:
                condition.append('Random')
            elif 'Predicted' in j:
                condition.append('Predicted')
            timeInd2 = (evokedO_dictAll['S' + str(i)][j].times > latencies2[0])*(evokedO_dictAll['S' + str(i)][j].times < latencies2[1])
        
            amp.append( np.max(evokedO_dictAll['S'+ str(i)][j].data[elecInd, timeInd2]))
            lats.append(latencies2[0] + np.argmax(evokedO_dictAll['S'+ str(i)][j].data[elecInd, timeInd2])/srate - 2.0)
            
            
    
d = {'Subject': subject, 'Condition': condition, 'Delay': delay, 'PeakLatency': lats, 'PeakAmplitude': amp}
dfO = pd.DataFrame(d)

In [ ]:
dfO

In [ ]:
sb.catplot(x = 'Delay', y = 'PeakAmplitude', data = dfO, hue = 'Condition', kind = 'point' )


sb.catplot(x = 'Delay', y = 'PeakLatency', data = dfO, hue = 'Condition', kind = 'point' )

In [ ]:
modelLMERT = smf.mixedlm("PeakAmplitude~Condition + Delay   ",dfO, groups = dfO['Subject'])
LMERTFit = modelLMERT.fit()
print(LMERTFit.summary())